In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install datasets
!pip install transformers
!pip install transformers[torch]
!pip install accelerate
!pip install rouge_score
!pip install evaluate

In [23]:
!pip install accelerate

  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)


In [3]:
# import transformers
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, Seq2SeqTrainingArguments
from datasets import load_dataset, load_from_disk, Dataset
from sklearn.model_selection import train_test_split
import accelerate
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import torch

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
data = pd.read_csv("/content/drive/My Drive/Copy of recipe_dataset_v2_5000.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [35]:
data.shape
# print(data['answer'][0])

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 1000
    })
})

In [ ]:
data = data[:2000]

def create_instruction(row):
    ner = ', '.join(eval(row['NER']))
    return f"I have {ner}. Give me some cooking ideas."

# Step 3: Define a function to create the 'response' column
def create_response(row):
    title = row['title']
    ingredients = ', '.join(eval(row['ingredients']))
    directions = ' '.join(eval(row['directions']))
    return f"you can make {title} with {ingredients}.\nHere's the instruction: {directions}"

# Step 4: Apply the functions to create new columns
data['question'] = data.apply(create_instruction, axis=1)
data['answer'] = data.apply(create_response, axis=1)

# Step 5: Select only the desired columns
reformatted_data = data[['question', 'answer']]

# Step 6: Save the reformatted data to a CSV file
reformatted_data.to_csv('reformatted_data.csv', index=False)

print(reformatted_data.head())

In [8]:
data = data[:2000]
data.tail(10)

,question,answer
1990,I have oleo eggs celery stalk onion green pepp...,you can make Scalloped Corn with 1 box Jiffy c...
1991,I have vanilla eggs light brown sugar pecans f...,you can make Blonde Brownies with 1/2 c . sof...
1992,I have instant tea sugar ground cloves ground ...,you can make Spiced Tea with 1 large pkg . le...
1993,I have shortening flour baking powder salt . G...,you can make Homemade Pot Pie with 2 c . flou...
1994,I have sugar eggs baking soda bananas water fl...,you can make Whole Wheat Banana Bread with 1/2...
1995,I have peanuts butterscotch ice cream topping ...,you can make Caramel Dip with 1 (8 oz . ) pkg ...
1996,I have cream of chicken soup sour cream turkey...,you can make Turkey Or Chicken Casserole with ...
1997,I have potatoes shin bone tomatoes boneless st...,you can make Peasant-Style Homemade Vegetable ...
1998,I have egg sugar water flour margarine yeast c...,you can make Bee Cake with 1 1/4 to 1 3/4 c . ...
1999,I have brown sugar molasses cooking oil water ...,you can make Swedish Baked Beans with 2 lb . ...


In [33]:
df = pd.DataFrame(data)

# df_random_1000 = df.sample(n=1000, random_state=42)
# df = df_random_1000
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

KeyError: 'train'

In [10]:
train_dataset

Dataset({
    features: ['question', 'answer', '__index_level_0__'],
    num_rows: 1600
})

In [37]:
# max_input = 512
# max_target = 128
# batch_size = 3
max_length = 512
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [12]:
def preprocess_function(examples):
    inputs = [q for q in examples['question']]
    model_inputs = tokenizer(inputs,padding='max_length', max_length=max_length, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['answer'], padding='max_length', max_length=max_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_dataset.map(preprocess_function, batched=True)
test_tokenized = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1600 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [24]:
train_tokenized

Dataset({
    features: ['question', 'answer', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1600
})

In [ ]:
# train_tokenized.save_to_disk('/content/drive/My Drive/train_tokenized')
# test_tokenized.save_to_disk ('/content/drive/My Drive/test_tokenized')

In [ ]:
# from datasets import load_from_disk

# # Load the dataset from the disk
# train_tokenized = load_from_disk('/content/drive/My Drive/train_tokenized')
# test_tokenized = load_from_disk('/content/drive/My Drive/test_tokenized')

In [38]:
# from transformers import Trainer, TrainingArguments
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer


training_args = Seq2SeqTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=2,   # batch size for training
    per_device_eval_batch_size=2,    # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    gradient_accumulation_steps=8,
    learning_rate=3e-4,
)

In [39]:
from datasets import load_metric

rouge = load_metric("rouge")

def compute_rouge(predictions, references):
    # Decode predictions and references if they are token IDs
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True, clean_up_tokenization_spaces=True) for pred in predictions.predictions]
    decoded_labels = [tokenizer.decode(label, skip_special_tokens=True, clean_up_tokenization_spaces=True) for label in references.label_ids]

    # Sometimes you have to remove padding or other special tokens manually
    decoded_labels = ["\n".join([ll for ll in label.splitlines() if len(ll) > 0]) for label in decoded_labels]

    # Compute ROUGE scores
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    # Extract a few results from ROUGE
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    return {"rouge": result}

/usr/local/lib/python3.10/dist-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# import nltk, evaluate
# import numpy as np
# from nltk.tokenize import sent_tokenize
# nltk.download("punkt", quiet=True)
# metric = evaluate.load("rouge")

# def compute_metrics(eval_preds):
#     preds, labels = eval_preds

#     # decode preds and labels
#     labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
#     decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
#     decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

#     # rougeLSum expects newline after each sentence
#     decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
#     decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

#     result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
#     return result


In [40]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_tokenized['train'],
    eval_dataset=dataset_tokenized['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [41]:
trainer.train()

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`labels` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
trainer.evaluate()

In [18]:
model.save_pretrained('/content/drive/My Drive/saved_model/bart1000_1')
tokenizer.save_pretrained('/content/drive/My Drive/saved_model/bart1000_1')


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


('/content/drive/My Drive/saved_model/bart1000_1/tokenizer_config.json',
 '/content/drive/My Drive/saved_model/bart1000_1/special_tokens_map.json',
 '/content/drive/My Drive/saved_model/bart1000_1/vocab.json',
 '/content/drive/My Drive/saved_model/bart1000_1/merges.txt',
 '/content/drive/My Drive/saved_model/bart1000_1/added_tokens.json')

In [21]:
# from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained('/content/drive/My Drive/saved_model/bart1000_1')
# tokenizer = BartTokenizer.from_pretrained('./saved_model/bart8000')

# Encode input context
# input_text = "what dishes can we make with sugar, chocolate, flour"
input_text = "I have chocolate what can I make"
inputs = tokenizer(input_text, return_tensors="pt")

# # Generate a response
# outputs = model.generate(inputs['input_ids'], num_beams=5, early_stopping=True)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Debug: Check input tokens
# print("Tokenized inputs:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))


outputs = model.generate(**inputs, max_length=512)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Debug: Check generated text
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))


[" canyou can make Corned Beef And Rice Casserole with 1 c.  flour, 1/2 tsp.  baking powder, 1 Tbsp.  salt, 1 tsp. baking soda, 1 1/4 c..  milk, 2 Tbsp butter or margarine. Here's the instruction : Mix flour, baking powder and salt together.  Add milk and butter.  Bake at 350° for 45 minutes. "]
